# An English man in New York

## Capstone Project: The Battle of Neighborhoods.
This is the final assignment of the Applied Data Science Capstone Course by IBM on Coursera.

## Intorduction

Introduction where you discuss the business problem and who would be interested in this project.

## Data

Data where you describe the data that will be used to solve the problem and the source of the data.

## Methodology

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

## Import required librarys

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#from bs4 import BeautifulSoup
import os

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

print('Libraries imported.')

## Results

Results section where you discuss the results.

## Discussion / Observations

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

## Conclusion

Conclusion section where you conclude the report.